In [ ]:
try:
  # This command only in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Get project files
!wget https://cdn.freecodecamp.org/project-data/cats-and-dogs/cats_and_dogs.zip

!unzip cats_and_dogs.zip

PATH = 'cats_and_dogs'

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')
test_dir = os.path.join(PATH, 'test')

# Get number of files in each directory. The train and validation directories
# each have the subdirecories "dogs" and "cats".
total_train = sum([len(files) for r, d, files in os.walk(train_dir)])
total_val = sum([len(files) for r, d, files in os.walk(validation_dir)])
total_test = len(os.listdir(test_dir))

# Variables for pre-processing and training.
batch_size = 128
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150

In [ ]:

train_image_generator = ImageDataGenerator(rescale = 1./255.)
validation_image_generator = ImageDataGenerator(rescale = 1./255.)
test_image_generator = ImageDataGenerator(rescale = 1./255.)


train_data_gen = train_image_generator.flow_from_directory(
    directory = train_dir,
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    batch_size = batch_size,
    class_mode="binary",
    shuffle=True,
    seed=42
)

val_data_gen = validation_image_generator.flow_from_directory(
    directory = validation_dir,
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    batch_size = batch_size,
    class_mode="binary",
    shuffle=True,
    seed=42
)



test_data_gen = test_image_generator.flow_from_directory(
        PATH,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=1,
        classes=['test'],
        class_mode=None,
        shuffle=False
)



In [ ]:
# 4
def plotImages(images_arr, probabilities = False):
    fig, axes = plt.subplots(len(images_arr), 1, figsize=(5,len(images_arr) * 3))
    if probabilities is False:
      for img, ax in zip( images_arr, axes):
          ax.imshow(img)
          ax.axis('off')
    else:
      for img, probability, ax in zip( images_arr, probabilities, axes):
          ax.imshow(img)
          ax.axis('off')
          if probability > 0.5:
              ax.set_title("%.2f" % (probability*100) + "% dog")
          else:
              ax.set_title("%.2f" % ((1-probability)*100) + "% cat")
    plt.show()

sample_training_images, _ = next(train_data_gen)
plotImages(sample_training_images[:5])


In [ ]:
train_image_generator = ImageDataGenerator(
    rotation_range=40,
    zoom_range=0.1,
    vertical_flip=False,
    horizontal_flip = True,
    rescale=1./255.,
    width_shift_range =0.1,
    height_shift_range =0.1,
    shear_range=0.1,
    brightness_range = [0.8, 1],
    fill_mode="nearest")


train_data_gen = train_image_generator.flow_from_directory(
    directory = train_dir,
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    # color_mode = "rgb",
    batch_size = batch_size,
    class_mode="binary",
    shuffle=True,
    # seed=42
)

In [ ]:
# 6
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                     directory=train_dir,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode='binary')

augmented_images = [train_data_gen[0][0][0] for i in range(5)]

plotImages(augmented_images)

In [ ]:
model = Sequential([
                  Conv2D(64, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
                  MaxPooling2D((2, 2)),
                  Conv2D(128, (3, 3), activation='relu'),
                  MaxPooling2D((2,2)),
                  Conv2D(64, (3, 3), activation='relu'),
                  Flatten(),
                  Dense(64, activation="relu"),
                  Dense(2, activation="sigmoid")
])

model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
    )

model.summary()

In [ ]:
epochs = 15

history = model.fit(train_data_gen,
                    validation_data=val_data_gen,
                    epochs=epochs,
                    steps_per_epoch=train_data_gen.n//train_data_gen.batch_size,
                    validation_steps=val_data_gen.n//val_data_gen.batch_size,
                    verbose=1
                    )

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5729: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


15/15 [==============================] - 260s 17s/step - loss: 0.8057 - accuracy: 0.5219 - val_loss: 0.6894 - val_accuracy: 0.5078
Epoch 2/15
15/15 [==============================] - 249s 17s/step - loss: 0.6948 - accuracy: 0.5000 - val_loss: 0.6924 - val_accuracy: 0.5212
Epoch 3/15
15/15 [==============================] - 245s 17s/step - loss: 0.6901 - accuracy: 0.5347 - val_loss: 0.6930 - val_accuracy: 0.5000
Epoch 4/15
15/15 [==============================] - 224s 15s/step - loss: 0.6926 - accuracy: 0.5134 - val_loss: 0.6933 - val_accuracy: 0.5056
Epoch 5/15
15/15 [==============================] - 239s 16s/step - loss: 0.6931 - accuracy: 0.5048 - val_loss: 0.6932 - val_accuracy: 0.4900
Epoch 6/15
15/15 [==============================] - 240s 16s/step - loss: 0.6931 - accuracy: 0.5021 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 7/15
15/15 [==============================] - 239s 16s/step - loss: 0.6930 - accuracy: 0.5203 - val_loss: 0.6927 - val_accuracy: 0.5346
Epoch 8/15
15/15 

KeyboardInterrupt: ignored

In [ ]:
# 9
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# 11
predictions = model.predict(test_data_gen)

answers =  [1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
            1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0,
            1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
            1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1,
            0, 0, 0, 0, 0, 0]

correct = 0

for probability, answer in zip(probabilities, answers):
  if round(probability) == answer:
    correct +=1

percentage_identified = (correct / len(answers)) * 100

passed_challenge = percentage_identified >= 63

print(f"Your model correctly identified {round(percentage_identified, 2)}% of the images of cats and dogs.")

if passed_challenge:
  print("You passed the challenge!")
else:
  print("You haven't passed yet. Your model should identify at least 63% of the images. Keep trying. You will get it!")